# Pre-process MSUPRP sample data for BATools

## Information about the dataset

The data comes from https://github.com/steibelj/gwaR, an example dataset for gwas study for MSUPRP dataset. The complete dataset can be also found at https://github.com/steibelj/GWA_growth.

The dataset comes as `gpData` from the `synbreed` R package.

This example will demonstrate how to create pedigree based relationship matrix and adaptive window based LD from the dataset.

Finally, we'll create the data file for BATools.

### 0. Before you start
Make sure you have the following R package installed: `dplyr`,`MCMCglmm`, `Matrix`

### 1. Explore the data 

In [1]:
load("MSUPRP_sample.RData")
class(MSUPRP_sample)
str(MSUPRP_sample)

[1] "gpData"

List of 7
 $ covar      :'data.frame':	253 obs. of  9 variables:
  ..$ id        : chr [1:253] "6070" "6071" "6086" "6088" ...
  ..$ phenotyped: logi [1:253] FALSE FALSE FALSE FALSE FALSE FALSE ...
  ..$ genotyped : logi [1:253] TRUE TRUE TRUE TRUE TRUE TRUE ...
  ..$ family    : logi [1:253] NA NA NA NA NA NA ...
  ..$ sex       : Factor w/ 2 levels "F","M": NA NA NA NA NA NA NA NA NA NA ...
  ..$ litter    : Factor w/ 142 levels "1","2","3","4",..: NA NA NA NA NA NA NA NA NA NA ...
  ..$ slgdt_cd  : Factor w/ 33 levels "1","2","3","4",..: NA NA NA NA NA NA NA NA NA NA ...
  ..$ age_slg   : int [1:253] NA NA NA NA NA NA NA NA NA NA ...
  ..$ car_wt    : num [1:253] NA NA NA NA NA NA NA NA NA NA ...
 $ pheno      : num [1:194, 1:3, 1] 5.22 5.19 5.44 5.36 5.25 5.29 5.44 5.26 5.35 5.39 ...
  ..- attr(*, "dimnames")=List of 3
  .. ..$ : chr [1:194] "1034" "1036" "1041" "1049" ...
  .. ..$ : chr [1:3] "ph_24h" "temp_24h" "driploss"
  .. ..$ : NULL
 $ geno       : num [1:251, 1:20597] 0 0 1